In [106]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import numpy as np
from dateutil.relativedelta import relativedelta

In [107]:
#numpy for SARIMA and Exponential

In [108]:
def handle_warning(message, category, filename, lineno, file=None, line=None):
    print(f"Warning: {message} for Item Code: {current_item_code}, Location: {current_location}")


1.Preparing Data by taking the sales history, criteria for forecasts and the iteminfo file.

In [109]:
sales_history_path = r'C:\Users\Anamind\Downloads\Abbott_SalesHistory5.xlsx'
input_file= pd.read_excel(sales_history_path)
criteria=r'C:\Users\Anamind\Downloads\Main Criteria.xlsx'
criteria=pd.read_excel(criteria)
item_info= r'C:\Users\Anamind\Downloads\Abbot_InputFiles\Abbott_ItemInfo.xlsx'
item_info=pd.read_excel(item_info, encoding='latin-1')

In [110]:
input_file

,Date,Quantity Sold,Item Code,Location,Channel
0,2022-06-30,486,2040001785,India,0


In [111]:
forecast_lessthan_18_months=pd.DataFrame()
merged_output=pd.DataFrame()
weights=pd.DataFrame()

Take the forecast horizon value mentioned as the number of months of forecast required.

In [112]:
unique_value = criteria['Forecast Horizon'].unique()
number = int(unique_value[0])

Suppose if there are any missing item codes in forecast dataframe but not in item info, add that item code and location to the forecast dataframe with the latest month and quantity sold as 0.

In [113]:
missing_item_codes = item_info[~item_info['Item Code'].isin(input_file['Item Code'])]

# Create a new dataframe with the same columns as input_file but only with the missing item codes and their locations
missing_items_df = pd.DataFrame(columns=input_file.columns)
missing_items_df['Item Code'] = missing_item_codes['Item Code']
missing_items_df['Location'] = missing_item_codes['Location']

# Drop columns that are all-NA from the missing_items_df
missing_items_df = missing_items_df.dropna(axis=1, how='all')

# Concatenate the missing items dataframe with the original input_file
input_file = pd.concat([input_file, missing_items_df], ignore_index=True)

In [114]:
input_file['Date'] = pd.to_datetime(input_file['Date'], dayfirst=True, errors='coerce')
latest_date = input_file['Date'].max()

In [115]:
input_file['Date'] = input_file['Date'].fillna(latest_date)
input_file = input_file.fillna(0)

In [116]:
input_file

,Date,Quantity Sold,Item Code,Location,Channel
0,2022-06-30,486.0,2040001785,India,0.0
1,2022-06-30,0.0,2040002300,India,0.0
2,2022-06-30,0.0,2040003482,India,0.0
3,2022-06-30,0.0,2040007225,India,0.0
4,2022-06-30,0.0,2040003089,India,0.0
5,2022-06-30,0.0,2040011024,India,0.0
6,2022-06-30,0.0,2040001786,India,0.0
7,2022-06-30,0.0,2040007220,India,0.0
8,2022-06-30,0.0,2040011297,India,0.0
9,2022-06-30,0.0,2040002040,India,0.0


Now let us take the first and last month present in the data. Make sure all the Item Code , Location combination is in the same month range and all the months in that range are present for each Item Code, Location combination. If its not present then add that month and year to that Item Code, Location giving Quantity Sold as 0 to maintain uniformity in all the Item Codes and Location.

In [117]:

input_file['Date'] = pd.to_datetime(input_file['Date'], format='%d-%m-%Y')

# Create 'month' column
input_file['month'] = input_file['Date'].dt.to_period('M')

# Find the earliest and latest months
earliest_month = input_file['month'].min()
latest_month = input_file['month'].max()

# Generate all combinations of unique item codes, locations, and channels
unique_combinations = input_file[['Item Code', 'Location', 'Channel']].drop_duplicates()

# Generate a DataFrame with a range of months from the earliest to the latest
months_range = pd.period_range(start=earliest_month, end=latest_month, freq='M')
months = pd.DataFrame({'month': months_range})

# Create a Cartesian product of months and unique combinations
months['key'] = 0
unique_combinations['key'] = 0
results= pd.merge(months, unique_combinations, on='key').drop(columns='key')

Lets aggregate the Quantity Sold values based on month, Item Code, Location, Channel

In [118]:
aggregated_input_data = input_file.groupby(['month', 'Item Code', 'Location', 'Channel'])['Quantity Sold'].sum().reset_index()
merged_data = pd.merge(results, aggregated_input_data, on=['month', 'Item Code', 'Location', 'Channel'], how='left')
merged_data['Quantity Sold'] = merged_data['Quantity Sold'].fillna(0)
sorted_data = merged_data.sort_values(by=['Item Code', 'Location', 'Channel']).reset_index(drop=True)

In [119]:
sorted_data

,month,Item Code,Location,Channel,Quantity Sold
0,2022-06,2040001785,India,0.0,486.0
1,2022-06,2040001786,India,0.0,0.0
2,2022-06,2040002040,India,0.0,0.0
3,2022-06,2040002300,India,0.0,0.0
4,2022-06,2040002320,India,0.0,0.0
5,2022-06,2040003089,India,0.0,0.0
6,2022-06,2040003170,India,0.0,0.0
7,2022-06,2040003482,India,0.0,0.0
8,2022-06,2040007220,India,0.0,0.0
9,2022-06,2040007225,India,0.0,0.0


In [120]:
merged_data

,month,Item Code,Location,Channel,Quantity Sold
0,2022-06,2040001785,India,0.0,486.0
1,2022-06,2040002300,India,0.0,0.0
2,2022-06,2040003482,India,0.0,0.0
3,2022-06,2040007225,India,0.0,0.0
4,2022-06,2040003089,India,0.0,0.0
5,2022-06,2040011024,India,0.0,0.0
6,2022-06,2040001786,India,0.0,0.0
7,2022-06,2040007220,India,0.0,0.0
8,2022-06,2040011297,India,0.0,0.0
9,2022-06,2040002040,India,0.0,0.0


Let us define all the forecasting models

In [121]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet

def forecast_simple_exponential_smoothing_no_trend_seasonality(ts, steps):
    model = ExponentialSmoothing(ts, trend=None, seasonal=None)
    results = model.fit()
    forecast = results.forecast(steps=steps)
    return forecast

def forecast_sarima(ts, steps):
    try:
        model = SARIMAX(ts, order=(0, 1, 0), seasonal_order=(1, 0, 0, 12))
        results = model.fit()
    except Exception as e:
        print("Encountered an error:", e)
        print("Changing seasonal_order to (0, 1, 0, 12)")
        model = SARIMAX(ts, order=(0, 1, 0), seasonal_order=(0, 1, 0, 12))
        results = model.fit()
    forecast = results.forecast(steps=steps)
    return forecast

def forecast_exponential_smoothing(ts, steps):
    initial_level = ts.iloc[0]
    initial_seasonal = np.zeros(12)
    model = ExponentialSmoothing(ts, seasonal='add', seasonal_periods=12, initialization_method='known', 
                                 initial_level=initial_level, initial_seasonal=initial_seasonal)
    results = model.fit()
    forecast = results.forecast(steps=steps)
    return forecast

def forecast_prophet(ts, steps):
    ts.index = pd.to_datetime(ts.index.to_timestamp())
    prophet_df = pd.DataFrame({'ds': ts.index, 'y': ts.values})
    model = Prophet()
    model.fit(prophet_df)
    future = model.make_future_dataframe(periods=steps, freq='M')
    forecast = model.predict(future)
    forecast_values = forecast.tail(steps)['yhat'].values
    return forecast_values


In [122]:
#numpybroadcasting

In [67]:
predictions_data

""


Lets initialise the functions of the forecasting models

In [68]:
#start_time = time.time()
predictions = []
grouped_df = sorted_data.groupby(['Item Code', 'Location', 'Channel'])

for (item_code, location, channel), group in grouped_df:
    
    if (group['Quantity Sold'] != 0).sum() >= 18:  # Adjusted for demonstration
        
        months_available = len(group)
        if months_available >= 1:
            first_nonzero_index = group['Quantity Sold'].ne(0).idxmax()
            ts = pd.Series(group['Quantity Sold'].values, index=group['month'])
            ts_current = ts[first_nonzero_index:]

            for i in range(6, 0, -1):  # Reduced for demonstration
                current_group = group.iloc[:-i]
                ts_current = pd.Series(current_group['Quantity Sold'].values, index=current_group['month'])

                sarima_forecast = forecast_sarima(ts_current, steps=i)
                exponential_smoothing_forecast = forecast_exponential_smoothing(ts_current, steps=i)

                actual_quantity_sold = group['Quantity Sold'].iloc[-i:].values

                # Ensure forecasts are non-negative using numpy broadcasting
                sarima_forecast = np.maximum(sarima_forecast, 0)
                exponential_smoothing_forecast = np.maximum(exponential_smoothing_forecast, 0)

                forecast_months = group['month'].iloc[-i:].values

                # Using numpy broadcasting to create the predictions list
                item_codes = np.repeat(item_code, i)
                locations = np.repeat(location, i)
                channels = np.repeat(channel, i)

                forecasts = np.column_stack([
                    item_codes,
                    locations,
                    channels,
                    sarima_forecast,
                    exponential_smoothing_forecast,
                    forecast_months,
                    actual_quantity_sold
                ])

                for forecast in forecasts:
                    predictions.append({
                        'Item code': forecast[0],
                        'Location': forecast[1],
                        'Channel': forecast[2],
                        f'sarima_forecast_{i}': forecast[3],
                        f'exponential_smoothing_forecast_{i}': forecast[4],
                        f'forecast_month_{i}': forecast[5],
                        f'actual_quantity_sold_{i}': forecast[6]
                    })

# Creating DataFrame outside the loop for efficiency
predictions_data = pd.DataFrame(predictions)

# Stopping the timer and calculating the elapsed time
#end_time = time.time()
#execution_time = end_time - start_time
#print("Execution time: {:.2f} seconds".format(execution_time))

In [69]:
predictions_data

""


In [70]:
import pandas as pd
import numpy as np

# Assuming predictions_data, sarima_total_abs_dev_data, and exponential_smoothing_total_abs_dev_data are defined

# Initialize lists to store total absolute deviations
sarima_total_abs_dev = []
exponential_smoothing_total_abs_dev = []
prophet_total_abs_dev = []

# Iterate through each forecast month
for i in range(1, 7):  # Assuming 6 forecast months
    # Calculate absolute deviation for SARIMA forecast
    predictions_data[f'sarima_abs_dev_{i}'] = abs(predictions_data[f'actual_quantity_sold_{i}'] - predictions_data[f'sarima_forecast_{i}'])

    # Calculate absolute deviation for Exponential Smoothing forecast
    predictions_data[f'exponential_smoothing_abs_dev_{i}'] = abs(predictions_data[f'actual_quantity_sold_{i}'] - predictions_data[f'exponential_smoothing_forecast_{i}'])

    # Calculate absolute deviation for Prophet forecast
    predictions_data[f'prophet_abs_dev_{i}'] = abs(predictions_data[f'actual_quantity_sold_{i}'] - predictions_data[f'prophet_forecast_{i}'])

    # Calculate total absolute deviation for SARIMA
    sarima_total_abs_dev.append(predictions_data.groupby(['Item code', 'Location', 'Channel'])[f'sarima_abs_dev_{i}'].sum())

    # Calculate total absolute deviation for Exponential Smoothing
    exponential_smoothing_total_abs_dev.append(predictions_data.groupby(['Item code', 'Location', 'Channel'])[f'exponential_smoothing_abs_dev_{i}'].sum())

    # Calculate total absolute deviation for Prophet
    prophet_total_abs_dev.append(predictions_data.groupby(['Item code', 'Location', 'Channel'])[f'prophet_abs_dev_{i}'].sum())

# Concatenate total absolute deviation data
sarima_total_abs_dev_data = pd.concat(sarima_total_abs_dev, axis=1).reset_index()
exponential_smoothing_total_abs_dev_data = pd.concat(exponential_smoothing_total_abs_dev, axis=1).reset_index()
prophet_total_abs_dev_data = pd.concat(prophet_total_abs_dev, axis=1).reset_index()

# Rename columns for SARIMA and Exponential Smoothing
sarima_total_abs_dev_data.columns = ['Item code', 'Location', 'Channel'] + [f'sarima_total_abs_dev_{i}' for i in range(1, 7)]
exponential_smoothing_total_abs_dev_data.columns = ['Item code', 'Location', 'Channel'] + [f'exponential_smoothing_total_abs_dev_{i}' for i in range(1, 7)]
prophet_total_abs_dev_data.columns = ['Item code', 'Location', 'Channel'] + [f'prophet_total_abs_dev_{i}' for i in range(1, 7)]

# Merge SARIMA and Exponential Smoothing total absolute deviation data
best_fit_data = pd.merge(sarima_total_abs_dev_data, exponential_smoothing_total_abs_dev_data, on=['Item code', 'Location', 'Channel'])

# Merge Prophet total absolute deviation data
best_fit_data = pd.merge(best_fit_data, prophet_total_abs_dev_data, on=['Item code', 'Location', 'Channel'])

# Calculate total absolute deviation sums
best_fit_data['sarima_total_abs_dev_sum'] = best_fit_data.iloc[:, 3:9].sum(axis=1)
best_fit_data['exponential_smoothing_total_abs_dev_sum'] = best_fit_data.iloc[:, 9:15].sum(axis=1)
best_fit_data['prophet_total_abs_dev_sum'] = best_fit_data.iloc[:, 15:21].sum(axis=1)

# Calculate weights based on absolute deviations
best_fit_data['sarima_ratio'] = best_fit_data['sarima_total_abs_dev_sum'] / (best_fit_data['sarima_total_abs_dev_sum'] + best_fit_data['exponential_smoothing_total_abs_dev_sum'] + best_fit_data['prophet_total_abs_dev_sum'])
best_fit_data['exponential_smoothing_ratio'] = best_fit_data['exponential_smoothing_total_abs_dev_sum'] / (best_fit_data['sarima_total_abs_dev_sum'] + best_fit_data['exponential_smoothing_total_abs_dev_sum'] + best_fit_data['prophet_total_abs_dev_sum'])
best_fit_data['prophet_ratio'] = best_fit_data['prophet_total_abs_dev_sum'] / (best_fit_data['sarima_total_abs_dev_sum'] + best_fit_data['exponential_smoothing_total_abs_dev_sum'] + best_fit_data['prophet_total_abs_dev_sum'])
best_fit_data['1_minus_sarima_ratio'] = 1 - best_fit_data['sarima_ratio']
best_fit_data['1_minus_exponential_smoothing_ratio'] = 1 -  best_fit_data['exponential_smoothing_ratio']
best_fit_data['1_minus_prophet_ratio'] = 1 -  best_fit_data['prophet_ratio']
total_sum_per_item = best_fit_data.groupby(['Item code', 'Location', 'Channel']).sum().reset_index()


total_sum_per_item['total_sum'] = total_sum_per_item['1_minus_sarima_ratio']  + total_sum_per_item['1_minus_exponential_smoothing_ratio']
total_sum_per_item['Sarima Weight'] = total_sum_per_item['1_minus_sarima_ratio'] / total_sum_per_item['total_sum']
total_sum_per_item['Exponential Smoothing Weight'] = total_sum_per_item['1_minus_exponential_smoothing_ratio'] / total_sum_per_item['total_sum']
total_sum_per_item['Prophet Weight'] = total_sum_per_item['1_minus_prophet_ratio'] / total_sum_per_item['total_sum']
# Select relevant columns for final weights
#weights = best_fit_data[['Item code', 'Location', 'Channel', 'sarima_ratio', 'exponential_smoothing_ratio', 'prophet_ratio']]
#weight = total_sum_per_item[['Item code','Location', 'Channel' ,'Sarima Weight','Exponential Smoothing Weight'  ]]
# Rename column 'Item code' to 'Item Code'
weight = total_sum_per_item[['Item code','Location', 'Channel' ,'Sarima Weight','Exponential Smoothing Weight', 'Prophet Weight'  ]]
weights = pd.DataFrame(weight)
weights = weights.rename(columns={'Item code': 'Item Code'})

# Display or further use weights DataFrame
print(weights)


KeyError: 'actual_quantity_sold_1'

In [71]:
weights

""


In [103]:
import pandas as pd
import numpy as np
import time
import warnings
from prophet import Prophet

# Assuming your data and functions are defined before this point
start_time = time.time()
grouped = sorted_data.groupby(['Item Code', 'Location', 'Channel'])
filtered_groups = {key: group for key, group in grouped if (group['Quantity Sold'] != 0).sum() >= 18}

forecasts = []

def handle_warning(message, category, filename, lineno, file=None, line=None):
    print(f"Warning: {message} for Item Code: {current_item_code}, Location: {current_location}")

# Register a custom warning handler
warnings.showwarning = handle_warning

for key, group in filtered_groups.items():
    item_code, location, channel = key
    current_item_code = item_code
    current_location = location
    
    historical_data = group.set_index('month')
    first_nonzero_index = historical_data['Quantity Sold'].ne(0).idxmax()
    test_data = historical_data.loc[first_nonzero_index:]
    
    if len(test_data) >= 18:
        sarima_forecast = forecast_sarima(test_data['Quantity Sold'], 18)
        exp_smoothing_forecast = forecast_exponential_smoothing(test_data['Quantity Sold'], 18)
        
        # Prophet forecasting
        prophet_forecast = forecast_prophet(test_data['Quantity Sold'], 18)
        
        # Calculate forecast index
        start_date = test_data.index[-1].to_timestamp() + pd.DateOffset(months=1)
        forecast_index = pd.date_range(start=start_date, periods=number, freq='M')  # Use 'M' for month-end
        
        # Convert forecast_index to string format
        forecast_index_str = forecast_index.strftime('%Y-%m-%d')
        
        # Apply numpy broadcasting for rounding and non-negative values
        sarima_forecast = np.maximum(0, np.round(sarima_forecast, 2))
        exp_smoothing_forecast = np.maximum(0, np.round(exp_smoothing_forecast, 2))
        prophet_forecast = np.maximum(0, np.round(prophet_forecast, 2))
        
        # Prepare forecasts in a structured format
        forecast_data = np.column_stack([
            np.repeat(item_code, number),
            np.repeat(location, number),
            np.repeat(channel, number),
            forecast_index_str,  # Use string format for dates
            sarima_forecast,
            exp_smoothing_forecast,
            prophet_forecast
        ])
        
        forecasts.extend(forecast_data)

columns = ['Item Code', 'Location', 'Channel', 'Month', 'SARIMA forecast', 'Exponential Smoothing forecast', 'Prophet forecast']
forecast_18months_history = pd.DataFrame(forecasts, columns=columns)

# Stopping the timer and calculating the elapsed time
end_time = time.time()
execution_time = end_time - start_time
print("Execution time: {:.2f} seconds".format(execution_time))


Execution time: 0.01 seconds


In [73]:
forecast_18months_history

,Item Code,Location,Channel,Month,SARIMA forecast,Exponential Smoothing forecast,Prophet forecast


In [74]:
forecast_18months_history

,Item Code,Location,Channel,Month,SARIMA forecast,Exponential Smoothing forecast,Prophet forecast


In [75]:
forecast_18months_history.to_csv('check1.csv')

In [76]:
def determine_best_fit(row):
    if row['Sarima Weight'] > row['Exponential Smoothing Weight'] and row['Sarima Weight'] > row['Prophet Weight']:
        return ('Sarima', row['SARIMA forecast'])
    elif row['Exponential Smoothing Weight'] > row['Sarima Weight'] and row['Exponential Smoothing Weight'] > row['Prophet Weight']:
        return ('Exponential Smoothing', row['Exponential Smoothing forecast'])
    else:
        return ('Prophet', row['Prophet forecast'])


In [77]:
forecast_18months_history['Location'] = forecast_18months_history['Location'].astype(str)
weights['Location'] = weights['Location'].astype(str)

forecast_18months_history['Channel'] = forecast_18months_history['Channel'].astype(str)
weights['Channel'] = weights['Channel'].astype(str)

# Convert 'Item Code' to string type in both dataframes if it's not already
forecast_18months_history['Item Code'] = forecast_18months_history['Item Code'].astype(str)
weights['Item Code'] = weights['Item Code'].astype(str)

# Merge forecast_18months_history with weights DataFrame based on 'Item Code', 'Location', and 'Channel'
merged_output = pd.merge(forecast_18months_history, weights, on=['Item Code','Location','Channel'])



# Apply the function to each row
merged_output['Best Fit Model'], merged_output['Best Fit Forecast'] = zip(*merged_output.apply(determine_best_fit, axis=1))

KeyError: 'Location'

In [78]:
merged_output

""


In [79]:
merged_output.to_csv('check1.csv')

In [81]:
forecast_lessthan_18_months

,Item Code,Location,Channel,Month,SARIMA forecast,Exponential Smoothing forecast,Prophet forecast


In [82]:
start_time = time.time()
grouped = sorted_data.groupby(['Item Code', 'Location', 'Channel'])
nonzero_count_mask = sorted_data.groupby(['Item Code', 'Location', 'Channel'])['Quantity Sold'].transform(lambda x: (x != 0).sum() > 1 and (x != 0).sum() < 18)

forecasts = []

def handle_warning(message, category, filename, lineno, file=None, line=None):
    print(f"Warning: {message} for Item Code: {current_item_code}, Location: {current_location}")

# Register a custom warning handler
warnings.showwarning = handle_warning

for key, group in filtered_groups.items():
    item_code, location, channel = key
    current_item_code = item_code
    current_location = location
    
    historical_data = group.set_index('month')
    first_nonzero_index = historical_data['Quantity Sold'].ne(0).idxmax()
    test_data = historical_data.loc[first_nonzero_index:]
    
    if len(test_data) >= 18:
        sarima_forecast = forecast_sarima(test_data['Quantity Sold'], 18)
        exp_smoothing_forecast = forecast_exponential_smoothing(test_data['Quantity Sold'], 18)
        
        # Prophet forecasting
        prophet_forecast = forecast_prophet(test_data['Quantity Sold'], 18)
        
        # Calculate forecast index
        start_date = test_data.index[-1].to_timestamp() + pd.DateOffset(months=1)
        forecast_index = pd.date_range(start=start_date, periods=18, freq='M')  # Use 'M' for month-end
        
        # Convert forecast_index to string format
        forecast_index_str = forecast_index.strftime('%Y-%m-%d')
        
        # Apply numpy broadcasting for rounding and non-negative values
        sarima_forecast = np.maximum(0, np.round(sarima_forecast, 2))
        exp_smoothing_forecast = np.maximum(0, np.round(exp_smoothing_forecast, 2))
        prophet_forecast = np.maximum(0, np.round(prophet_forecast, 2))
        
        # Prepare forecasts in a structured format
        forecast_data = np.column_stack([
            np.repeat(item_code, 18),
            np.repeat(location, 18),
            np.repeat(channel, 18),
            forecast_index_str,  # Use string format for dates
            sarima_forecast,
            exp_smoothing_forecast,
            prophet_forecast
        ])
        
        forecasts.extend(forecast_data)



columns = ['Item Code', 'Location', 'Channel', 'Month', 'SARIMA forecast', 'Exponential Smoothing forecast','Prophet Forecast']
forecast_lessthan_18_months= pd.DataFrame(forecasts, columns=columns)

# Stopping the timer and calculating the elapsed time
end_time = time.time()
execution_time = end_time - start_time
print("Execution time: {:.2f} seconds".format(execution_time))

Execution time: 0.03 seconds


In [83]:

forecast_lessthan_18_months['Best Fit Forecast'] = forecast_lessthan_18_months['Exponential Smoothing forecast']
forecast_lessthan_18_months['Best Fit Model'] = 'Exponential Smoothing'
#merged_output.drop(['Sarima Weight', 'Exponential Smoothing Weight','Prophet Weight'], axis=1, inplace=True)
Result= pd.concat([merged_output, forecast_lessthan_18_months], ignore_index=True)

In [84]:
forecast_lessthan_18_months

,Item Code,Location,Channel,Month,SARIMA forecast,Exponential Smoothing forecast,Prophet Forecast,Best Fit Forecast,Best Fit Model


In [123]:
one_nonzero_combinations = []

# Iterate over each unique combination of 'Item Code', 'Location', and 'Channel'
for key, group in sorted_data.groupby(['Item Code', 'Location', 'Channel']):
    item_code, location, channel = key

    # Count the number of non-zero quantity sold values
    nonzero_count = (group['Quantity Sold'] != 0).sum()

    # Check if there's only one non-zero quantity sold value
    if nonzero_count == 1:
        one_nonzero_combinations.append(key)

filtered_output = sorted_data[sorted_data.set_index(['Item Code', 'Location', 'Channel']).index.isin(one_nonzero_combinations)]

forecasts3 = []

for key, group in filtered_output.groupby(['Item Code', 'Location', 'Channel']):
    item_code, location, channel = key

    # Get historical data for the current combination
    historical_data = group.set_index('month')

    # Check if there is at least one non-zero value in historical data
    if (historical_data['Quantity Sold'] != 0).any():
        # Get the most recent non-zero month
        last_nonzero_month = historical_data[historical_data['Quantity Sold'] != 0].index[-1]

        # Check if the most recent non-zero month is the last available data point
        if historical_data.index[-1] == last_nonzero_month:
            # If only one month of non-zero data and it's the most recent month,
            # repeat the last month's value for Exponential Smoothing forecast
            exp_smoothing_value = historical_data.iloc[-1]['Quantity Sold']
            # Set SARIMA forecast value to 0
            sarima_value = 0
            prophet_forecast = 0
        else:
            # If there is non-zero data elsewhere, run both forecasts
            # Calculate SARIMA forecast value here (to be implemented)
            sarima_value = 0  # Placeholder, implement SARIMA forecasting here
            exp_smoothing_value = 0 
            prophet_forecast = 0
    else:
        # If all historical data points are zero, set both forecasts to 0
        exp_smoothing_value = 0
        sarima_value = 0
        prophet_forecast = 0
    # Generate forecast index for 12 months
    start_date = historical_data.index[-1].to_timestamp() + relativedelta(months=1)
    forecast_index = pd.date_range(start=start_date, periods= number, freq='M')

    # Repeat the last month's value for Exponential Smoothing forecast for 12 months
    exp_smoothing_forecast = [exp_smoothing_value] * number

    # Create forecasts for 12 months
    for i in range(18):
        forecasts3.append([item_code, location, channel, forecast_index[i], sarima_value, exp_smoothing_forecast[i],prophet_forecast])

columns = ['Item Code', 'Location', 'Channel', 'Month', 'SARIMA forecast', 'Exponential Smoothing forecast','Prophet Forecast']
forecast_12months = pd.DataFrame(forecasts3, columns=columns)

# Adding Best Fit Forecast and Best Fit Model columns
forecast_12months['Best Fit Forecast'] = forecast_12months['Exponential Smoothing forecast']
forecast_12months['Best Fit Model'] = 'Exponential Smoothing'
OutputFile = pd.concat([Result, forecast_12months], ignore_index=True)

In [125]:
OutputFile

,Item Code,Location,Channel,Month,SARIMA forecast,Exponential Smoothing forecast,Prophet Forecast,Best Fit Forecast,Best Fit Model
0,2040001785,India,0.0,2022-07-31,0,486.0,0,486.0,Exponential Smoothing
1,2040001785,India,0.0,2022-08-31,0,486.0,0,486.0,Exponential Smoothing
2,2040001785,India,0.0,2022-09-30,0,486.0,0,486.0,Exponential Smoothing
3,2040001785,India,0.0,2022-10-31,0,486.0,0,486.0,Exponential Smoothing
4,2040001785,India,0.0,2022-11-30,0,486.0,0,486.0,Exponential Smoothing
5,2040001785,India,0.0,2022-12-31,0,486.0,0,486.0,Exponential Smoothing
6,2040001785,India,0.0,2023-01-31,0,486.0,0,486.0,Exponential Smoothing
7,2040001785,India,0.0,2023-02-28,0,486.0,0,486.0,Exponential Smoothing
8,2040001785,India,0.0,2023-03-31,0,486.0,0,486.0,Exponential Smoothing
9,2040001785,India,0.0,2023-04-30,0,486.0,0,486.0,Exponential Smoothing


In [126]:
zero_quantity = sorted_data.groupby(['Item Code', 'Location', 'Channel']).filter(lambda x: (x['Quantity Sold'] == 0).all())
last_month_year = zero_quantity['month'].max()
start_date = pd.Period(last_month_year, freq='M') + 1  # Get the next month
forecast_index = pd.period_range(start=start_date, periods=number, freq='M')  # 12 months forecast

# Drop 'Quantity Sold' column
zero_quantity = zero_quantity.drop(columns=['Quantity Sold'])

# Add new columns with default values
zero_quantity['SARIMA forecast'] = 0
zero_quantity['Exponential Smoothing forecast'] = 0
zero_quantity['Prophet'] = 0
zero_quantity['Best Fit Model'] = 'Exponential Smoothing'
zero_quantity['Best Fit Forecast'] = 0

# Create an empty DataFrame to store the forecast index
forecast_zero = pd.DataFrame(columns=zero_quantity.columns)

# Iterate over each period in the forecast index and append it to the DataFrame
for period in forecast_index:
    zero_quantity_copy = zero_quantity.copy()
    zero_quantity_copy['month'] = period
    forecast_zero = pd.concat([forecast_zero, zero_quantity_copy], ignore_index=True) 
forecast_zero = forecast_zero.drop_duplicates() 
forecast_zero['Month'] = forecast_zero['month'].dt.to_timestamp() + pd.offsets.MonthEnd(0)  
forecast_zero = forecast_zero.drop(columns=['month'])  
OutputFile= pd.concat([OutputFile, forecast_zero], ignore_index=True)

In [127]:
forecast_zero

,Item Code,Location,Channel,SARIMA forecast,Exponential Smoothing forecast,Prophet,Best Fit Model,Best Fit Forecast,Month
0,2040001786,India,0.0,0,0,0,Exponential Smoothing,0,2022-07-31
1,2040002040,India,0.0,0,0,0,Exponential Smoothing,0,2022-07-31
2,2040002300,India,0.0,0,0,0,Exponential Smoothing,0,2022-07-31
3,2040002320,India,0.0,0,0,0,Exponential Smoothing,0,2022-07-31
4,2040003089,India,0.0,0,0,0,Exponential Smoothing,0,2022-07-31
...,...,...,...,...,...,...,...,...,...
337,2040011297,India,0.0,0,0,0,Exponential Smoothing,0,2023-12-31
338,2040011444,India,0.0,0,0,0,Exponential Smoothing,0,2023-12-31
339,2040011445,India,0.0,0,0,0,Exponential Smoothing,0,2023-12-31
340,2040011446,India,0.0,0,0,0,Exponential Smoothing,0,2023-12-31


In [102]:
forecast_zero.to_csv('check1.csv')

In [95]:
OutputFile

,Item Code,Location,Channel,Month,SARIMA forecast,Exponential Smoothing forecast,Prophet Forecast,Best Fit Forecast,Best Fit Model
0,2040001785,India,0.0,2022-07-31,0,486.0,NaN,486.0,Exponential Smoothing
1,2040001785,India,0.0,2022-08-31,0,486.0,NaN,486.0,Exponential Smoothing
2,2040001785,India,0.0,2022-09-30,0,486.0,NaN,486.0,Exponential Smoothing
3,2040001785,India,0.0,2022-10-31,0,486.0,NaN,486.0,Exponential Smoothing
4,2040001785,India,0.0,2022-11-30,0,486.0,NaN,486.0,Exponential Smoothing
...,...,...,...,...,...,...,...,...,...
235,2040011297,India,0.0,2023-06-30,0,0,NaN,0,Exponential Smoothing
236,2040011444,India,0.0,2023-06-30,0,0,NaN,0,Exponential Smoothing
237,2040011445,India,0.0,2023-06-30,0,0,NaN,0,Exponential Smoothing
238,2040011446,India,0.0,2023-06-30,0,0,NaN,0,Exponential Smoothing
